In [12]:
df.head()

,CLIENTNUM,Attrition Flag,Customer Age,Gender,Dependent count,Education Level,Marital Status,Income Category,Card Category,Months on book,...,Credit Limit,Total Revolving Bal,Avg Open To Buy,Total Amt Chng Q4 Q1,Total Trans Amt,Total Trans Ct,Total Ct Chng Q4 Q1,Avg Utilization Ratio,Credit Utilization,Limit Income Ratio
0,712396908,Existing Customer,57,F,2,Graduate,Married,38309,Blue,48,...,2436.0,680,1756.0,1.190,1570,29,0.611,0.279,0.279146,0.063588
1,708508758,Attrited Customer,62,F,0,Graduate,Married,28326,Blue,49,...,1438.3,0,1438.3,1.047,692,16,0.600,0.000,0.000000,0.050777
2,811604133,Existing Customer,47,F,4,Unknown,Single,31855,Blue,36,...,2492.0,1560,932.0,0.573,1126,23,0.353,0.626,0.626003,0.078229
3,789124683,Existing Customer,54,M,2,Unknown,Married,96145,Blue,42,...,12217.0,0,12217.0,1.075,1110,21,0.750,0.000,0.000000,0.127068
4,771071958,Existing Customer,41,F,3,Graduate,Single,38894,Blue,28,...,7768.0,1669,6099.0,0.797,1051,22,0.833,0.215,0.214856,0.199722


In [13]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5298 entries, 0 to 5297
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 5298 non-null   int64  
 1   Attrition Flag            5298 non-null   object 
 2   Customer Age              5298 non-null   int64  
 3   Gender                    5298 non-null   object 
 4   Dependent count           5298 non-null   int64  
 5   Education Level           5298 non-null   object 
 6   Marital Status            5298 non-null   object 
 7   Income Category           5298 non-null   int64  
 8   Card Category             5298 non-null   object 
 9   Months on book            5298 non-null   int64  
 10  Total Relationship Count  5298 non-null   int64  
 11  Months Inactive 12 mon    5298 non-null   int64  
 12  Contacts Count 12 mon     5298 non-null   int64  
 13  Credit Limit              5298 non-null   float64
 14  Total Re

---

In [10]:
# Install CatBoost (run once in Colab)
!pip install catboost openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.5 MB/s eta 0:00:00


In [11]:
# 2. Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [13]:
# 3. Load and preprocess the dataset
# . Load the dataset
df = pd.read_excel("DS1-BANK-CHURNERS-FINAL-DATASET.xlsx")

In [14]:
#   3a. Drop identifier column
df.drop(columns=['CLIENTNUM'], inplace=True)

#   3b. Encode the target variable
df['Attrition_Flag'] = df['Attrition Flag'].map({
    'Existing Customer': 0,
    'Attrited Customer': 1
})
df.drop(columns=['Attrition Flag'], inplace=True)

#   3c. Encode remaining categorical features
cat_cols = df.select_dtypes(include='object').columns.tolist()
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# 4. Define features and target
X = df.drop(columns=['Attrition_Flag'])
y = df['Attrition_Flag']

# 5. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [15]:
# 6. Train CatBoost classifier
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    eval_metric='F1',
    auto_class_weights='Balanced',
    verbose=0,
    random_seed=42
)
model.fit(X_train, y_train, cat_features=cat_cols)

# 7. Make predictions
y_pred = model.predict(X_test)

In [16]:
# 8. Evaluate performance
print("🔍 Model Performance on Test Set:")
print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall   : {recall_score(y_test, y_pred):.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Existing Customer", "Attrited Customer"]))

🔍 Model Performance on Test Set:
Accuracy : 0.9717
Precision: 0.9077
Recall   : 0.9248

Classification Report:
                   precision    recall  f1-score   support

Existing Customer       0.98      0.98      0.98      1324
Attrited Customer       0.91      0.92      0.92       266

         accuracy                           0.97      1590
        macro avg       0.95      0.95      0.95      1590
     weighted avg       0.97      0.97      0.97      1590



In [2]:
# Install required library if not already
!pip install catboost openpyxl

# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

# Load dataset from your Google Drive path
df = pd.read_excel("DS1-BANK-CHURNERS-FINAL-DATASET.xlsx")

# Drop client ID and encode target
df.drop(columns=['CLIENTNUM'], inplace=True)
df['Attrition_Flag'] = df['Attrition Flag'].map({'Existing Customer': 0, 'Attrited Customer': 1})
df.drop(columns=['Attrition Flag'], inplace=True)

# Encode categorical features
cat_cols = df.select_dtypes(include='object').columns.tolist()
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

# Split into training and test sets
X = df.drop(columns=['Attrition_Flag'])
y = df['Attrition_Flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Train the CatBoost model
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    eval_metric='F1',
    auto_class_weights='Balanced',
    verbose=0,
    random_seed=42
)
model.fit(X_train, y_train, cat_features=cat_cols)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.1 MB/s eta 0:00:00


In [4]:
# Make predictions and evaluate
y_pred = model.predict(X_test)

print("🔍 Model Performance on Test Set:")
print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall   : {recall_score(y_test, y_pred):.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Existing Customer", "Attrited Customer"]))


🔍 Model Performance on Test Set:
Accuracy : 0.9642
Precision: 0.9035
Recall   : 0.8797

Classification Report:
                   precision    recall  f1-score   support

Existing Customer       0.98      0.98      0.98      1324
Attrited Customer       0.90      0.88      0.89       266

         accuracy                           0.96      1590
        macro avg       0.94      0.93      0.93      1590
     weighted avg       0.96      0.96      0.96      1590

